# Examples how to use scripts in this repository

In [ ]:
from samplers import *
from solvers import *
from plots import  *
from matplotlib import pylab
from pylab import rcParams
rcParams['figure.figsize'] = 5, 3

# Single run of ALS algorithm
Below are two examples of generating plynomials, using a solver and plotting the resutls.

In the first example the distance from the camera is equal to the assumed distance, and in the second case it is different. 

In [ ]:
start_param = [6, -10, 5, 0.5]
N = 2*(len(start_param))
assumed_distance = 1
true_distance = assumed_distance
polynomial = SecondSurfacePolynomial(start_param)
plot_results(polynomial, 'k', lw=1, label='polynomial')
sampler = SurfaceSampler(polynomial, N, [-0.6, true_distance, assumed_distance], interval_length=1, sigma=0.0)
sample_values = sampler.sample_values; 

start_pos = SecondSurfacePolynomial.shifted_positions(sampler.sample_positions, trace_param=[0, 1, assumed_distance])
stem_results(start_pos, sample_values, 'k', label="start. positions")

solver = ConstrainedALS(
    sample_values,
    polynomial.model_size,
    SecondSurfacePolynomial,
    start_pos=sampler.sample_positions,
    stopping_error=1e-10,
    beta=1,
    show_plots=False,
    max_iter=10000,
    fl=assumed_distance
    )
solver.solve()

stem_results(solver.get_position_estimates(), solver.get_samples(), 'g', label="positions found")
polynomial = FullSurfacePolynomial(solver.parameter_estimate)
plot_results(polynomial, 'g', lw =1, label='recovered')

pylab.title("initialised with correct scale", fontsize=10)
pylab.xlabel(r'$t$', fontsize=10)
pylab.legend(loc="lower right", fontsize=10)
pylab.gcf().subplots_adjust(bottom=0.15)
pylab.show()

In [ ]:
start_param = [6, -10, 5, 0.5]
N = 2*(len(start_param))
assumed_distace = 1.0
true_distance = 1.2
polynomial = SecondSurfacePolynomial(start_param)
plot_results(polynomial,'k',lw=1, label='polynomial')
sampler = SurfaceSampler(polynomial,N,[-0.6, true_distance, assumed_distance], interval_length=1, sigma=0.0)
sample_values = sampler.sample_values; 

start_pos = SecondSurfacePolynomial.shifted_positions(sampler.sample_positions,trace_param=[0, 1, assumed_distace])
stem_results(start_pos,sample_values,'k',label="start. positions")

solver = ConstrainedALS(
    sample_values,
    polynomial.model_size,
    SecondSurfacePolynomial,
    start_pos=sampler.sample_positions,
    stopping_error=1e-10,
    beta=1,
    show_plots=False,
    max_iter=10000,
    fl=assumed_distance
    )
solver.solve()

stem_results(solver.get_position_estimates(),solver.get_samples(),'g',label="positions found")
polynomial = FullSurfacePolynomial(solver.parameter_estimate)
plot_results(polynomial,'g',lw =1, label='recovered')

pylab.title("initialised with incorrect scale", fontsize=10)
pylab.xlabel(r'$t$',fontsize=10)
pylab.legend(loc="lower right", fontsize=10)
pylab.gcf().subplots_adjust(bottom=0.15)
pylab.show()

# Running many numerical experiments
Numerical experiments are running based on a paramteters written down in the sorted set.
The script `prepare_tests` generates and saves such set. Then the script `surface_experiments` loads the set of parameters and run actuall experiments. 

Below is example how to generate set of experiment parameters and then run the experiments, and then finally plot them. 

In [ ]:
import pickle
import numpy.random as nr
import numpy as np
from sortedcontainers import SortedSet
from surface_experiments import *
from plots import *

### Prepare experiments
Create set of parameters in the same format as used in the `surface_experiments` sript.

In [ ]:
test_set = SortedSet([])
all_degrees = SortedSet([])
new_params = True
n_tests = 2

noises = np.linspace(-1, 0, 10)

for polynomial_degree in [5]:
    for ovs in [1, 2, 4, 8]:
        for noise_scale in noises:
            test_set.add((polynomial_degree, ovs, noise_scale))
            all_degrees.add(polynomial_degree)

if new_params:
    for polynomial_degree in all_degrees:
        params = nr.randn(n_tests, polynomial_degree)
        params[:, 0] = 0.5
        np.savetxt("example_polynomials" + str(polynomial_degree) + ".csv", params, delimiter=",")


### Run experiments

Load set of paramters and run experiments in parallel.

In [ ]:
save = True
plots = False
verbose = False
n_tests = 2  # number of tests (should be at least two)
directory = "example_results/"
f = 1.0  # distance between the origin and the image plane
b = 1.0  # intersection between camera axis and the surface
slopes = np.linspace(-np.pi / 9, np.pi / 9, 13)


def test_block_unpack(t):
    return test_block(t[0], t[1], t[2], n_tests, slopes, verbose, save, directory, b, f, plots)


for t in test_set:
    print(t)

# start 4 worker processes
pool = Pool(processes=4)
pool.map(test_block_unpack, test_set);

### Plot results 

In [ ]:
n_pow = np.linspace(-1, 0, 10)
n_len = len(n_pow)

results = np.empty((2, 4, n_len))
all_errors = np.empty((4, n_len, n_tests*13))
all_power = np.empty((4, n_len, n_tests*13))
n = 5

for (overs, idx) in zip([1, 2, 4, 8], range(4)):
    for (nl, n_idx) in zip(n_pow, range(len(n_pow))):
        code = code = "{}_{}_{:.2f}".format(n, overs,nl)
        power = np.load('example_results/pow_' + code + '.npy')
        errors = np.load('example_results/errors_' + code + '.npy')
        errors[np.isnan(errors)]=1
        results[0, idx, n_idx] = np.degrees(np.percentile(errors, q=50))
        results[1, idx, n_idx] = np.degrees(np.mean(errors.flatten()))
        all_power[idx, n_idx,:] = power.flatten()
        all_errors[idx, n_idx,:] = errors.flatten()
        
snr = 10*(np.log(np.mean(all_power.flatten())) + 2 * n_pow)
beg = 0
fig = plt.figure()
plt.semilogy(snr[beg:], results[0,0,beg:].T, "-.")
plt.semilogy(snr[beg:], results[1,0,beg:].T)
plt.semilogy(snr[beg:], results[0,3,beg:].T, "--")
plt.semilogy(snr[beg:], results[1,3,beg:].T, ":")
plt.xlabel(r"$SNR_{dB}$")
plt.ylabel("error (in degrres)")
plt.legend([r"median error (no overs.)", r"mean error (no overs.)", r"median error 8$\times$ overs.", r"mean error 8$\times$ overs."], loc='best')
plt.grid()
plt.tight_layout()
plt.show()
fig.savefig('snr2.pdf')